# Detect Prohibited Applications Spawning cmd exe Unit Test

#### This search looks for executions of cmd.exe spawned by a process that is often abused by attackers and that does not typically launch cmd.exe. This is a SPL2 implementation of the rule `Detect Prohibited Applications Spawning cmd.exe`

Source: https://github.com/splunk/security-content/blob/develop/detections/endpoint/prohibited_apps_spawning_cmdprompt___ssa.yml

In [92]:
import json
data='{"process_path":"c:\\\windows\\\system32","process_name":"cmd.exe","process":"C:\\\Windows\\\system32\\\cmd.exe","parent_process_name":"C:\\\Program Files\\\Microsoft Office\\\winword.exe","dest_user_id":"eyJlbnRpdHlUeXBlIjoiVVNFUiIsInByaW1hcnlBcnRpZmFjdCI6ImFkbWluaXN0cmF0b3IiLCJwcmltYXJ5QXJ0aWZhY3RUeXBlIjoiV0lORE9XU19BQ0NPVU5UX05BTUUifQ","dest_device_id":"eyJlbnRpdHlUeXBlIjoiREVWSUNFIiwicHJpbWFyeUFydGlmYWN0Ijoid2luLWRjLTY1NjUwNzEiLCJwcmltYXJ5QXJ0aWZhY3RUeXBlIjoiRE5TIn0","_time":"1602004409000"}'
json.loads(data)
with open("detect_prohibited_applications_spawning_cmd_exe.json", "w") as outfile:
    outfile.write(data)

In [94]:
| from read_text("s3://smle-experiments/datasets/ssa/detect_prohibited_applications_spawning_cmd_exe.json")
| eval input_event=from_json_object(value)

| eval timestamp=parse_long(ucast(map_get(input_event, "_time"), "string", null))
| eval process_name=ucast(map_get(input_event, "process_name"), "string", null),
parent_process=lower(ucast(map_get(input_event, "parent_process_name"), "string", null)),
dest_user_id=ucast(map_get(input_event, "dest_user_id"), "string", null),
dest_device_id=ucast(map_get(input_event, "dest_device_id"), "string", null)
| where process_name="cmd.exe"
| rex field=parent_process "(?<field0>[^\\\\]+)$"
| where field0="winword.exe" OR
        field0="excel.exe" OR
        field0="outlook.exe" OR
        field0="powerpnt.exe" OR
        field0="visio.exe" OR
        field0="mspub.exe" OR
        field0="acrobat.exe" OR
        field0="acrord32.exe" OR
        field0="chrome.exe" OR
        field0="iexplore.exe" OR
        field0="opera.exe" OR
        field0="firefox.exe" OR
        field0="java.exe" OR
        field0="powershell.exe"
| eval start_time=timestamp,
end_time=timestamp,
entities=mvappend(dest_device_id, dest_user_id),
body="TBD";

 Finished.                     

,start_time,dest_device_id,field0,entities,process_name,end_time,dest_user_id,parent_process,body,value,input_event,timestamp
0,2020-10-06 17:13:29,eyJlbnRpdHlUeXBlIjoiREVWSUNFIiwicHJpbWFyeUFydG...,winword.exe,[eyJlbnRpdHlUeXBlIjoiREVWSUNFIiwicHJpbWFyeUFyd...,cmd.exe,2020-10-06 17:13:29,eyJlbnRpdHlUeXBlIjoiVVNFUiIsInByaW1hcnlBcnRpZm...,c:\program files\microsoft office\winword.exe,TBD,"{""process_path"":""c:\\windows\\system32"",""proce...","{'process_path': 'c:\windows\system32', 'proce...",2020-10-06 17:13:29
